# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [13]:
## SETUP used later

import sys
import os
import json
import pandas as pd
sys.path.insert(1, '/locale/data/jupyter/prando/wd-project/src')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-horror_franchises-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Horror Franchises explorative search") 

Consider the following exploratory scenario:


> we are interested in horror movies, in particular we care about the slasher subgenre, and we want to compare the Halloween and the Friday the 13th francise. 

Background knowledge useful for better conducting the search: Halloween is a movie from 1978 with many sequels and the set of Halloween + all the sequels is called the "Halloween franchise". The same considerations hold for Friday the 13th.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q221103`     | Halloween       | node |
| `wd:Q1243029`| Friday the 13th       | node |





Also consider

```
wd:Q1243029 ?p ?obj .
```

is the BGP to retrieve all **properties of Friday the 13th**

The workload should


1. Investigate the workers of the two original movies and check any commonality 

2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

3. Check which actor/actress did more movies after the release of these two films (consider the films separately)

4. Get all the movies of the two franchises (return year and director)

5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)? 

6. Investigate which franchise is highly regarded by the critics overall

7. Investigate which franchise costed more 

In [14]:
## startup the evaluation
# setup the file and create the empty json
ipname = "horror_franchises.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 5_2


### Task 1: Investigate the workers of the two original movies and check any commonality

Find the filmmaking occupation class

In [15]:
# find workers in a film
queryString = """
SELECT DISTINCT ?work ?pname
WHERE { 

wd:Q1243029 ?p ?obj .
?obj wdt:P31 wd:Q5;
    wdt:P106 ?job.
?job wdt:P31 ?work.

#Labels
?work sc:name ?pname .
} 
"""

print("Results")
x=run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q12737077'), ('pname', 'occupation')]
[('work', 'http://www.wikidata.org/entity/Q28640'), ('pname', 'profession')]
[('work', 'http://www.wikidata.org/entity/Q108300140'), ('pname', 'occupation group according to ISCO')]
[('work', 'http://www.wikidata.org/entity/Q66715801'), ('pname', 'musical profession')]
[('work', 'http://www.wikidata.org/entity/Q4220920'), ('pname', 'filmmaking occupation')]
[('work', 'http://www.wikidata.org/entity/Q15839299'), ('pname', 'theatrical occupation')]
6


In [16]:
# compare all workers 
queryString = """
SELECT DISTINCT ?film ?name COUNT(DISTINCT ?worker) AS ?workers
WHERE { 
    VALUES ?film {wd:Q1243029 wd:Q221103}
    ?film ?p ?worker .
    ?worker wdt:P31 wd:Q5 ;
            wdt:P106/wdt:P31 wd:Q4220920 .

    #Labels
    ?film sc:name ?name .
}
GROUP BY ?film ?name
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('workers', '13')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('workers', '15')]
2


In [17]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 5_2
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_2.json
JSON object updated


In [7]:
# check if someone worked on both films
queryString = """
SELECT DISTINCT ?worker ?name
WHERE { 
    VALUES ?film {wd:Q1243029 wd:Q221103}
    ?film ?p ?worker .
    ?worker wdt:P31 wd:Q5 ;
            wdt:P106/wdt:P31 wd:Q4220920 .

    #Labels
    ?worker sc:name ?name .
}
GROUP BY ?worker ?name
HAVING (COUNT(DISTINCT ?film)>1)
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
# compare cast member
queryString = """
SELECT DISTINCT ?film ?name COUNT(?worker) AS ?cast
WHERE { 
    VALUES ?film {wd:Q1243029 wd:Q221103}
    ?film wdt:P161 ?worker .

    #Labels
    ?film sc:name ?name .
}
GROUP BY ?film ?name
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('cast', '9')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('cast', '13')]
2


In [10]:
# compare countries 
queryString = """
SELECT ?film ?name ?country ?cname COUNT(DISTINCT ?worker) AS ?howMany
WHERE { 
    VALUES ?film {wd:Q1243029 wd:Q221103}
    ?film ?p ?worker .
    ?worker wdt:P31 wd:Q5 ;
            wdt:P27 ?country;
            wdt:P106/wdt:P31 wd:Q4220920 .

    #Labels
    ?film sc:name ?name .
    ?country sc:name ?cname .
}
GROUP BY ?film ?name ?country ?cname
ORDER BY DESC (?howMany)
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('howMany', '14')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('howMany', '12')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('country', 'http://www.wikidata.org/entity/Q858'), ('cname', 'Syria'), ('howMany', '1')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '1')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '1')]
5


### Task 2: Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release. 

To answer this question we need to find the time of the release of the two films and to define the criteria to determine how much an actor/actress is famous.
In this case we simply use the number of films where the actor/actress acted.

In [11]:
#count in how many films the actors of friday 13th and halloween acted before the release, counting them separately
queryString = """
SELECT ?f ?name ?actor ?aName (COUNT(DISTINCT ?film) AS ?films)
WHERE { 
    # friday 13th and Halloween
    VALUES ?f {wd:Q1243029 wd:Q221103}.
    ?f wdt:P161 ?actor;
                wdt:P577 ?datef.
    ?film wdt:P161 ?actor;
          wdt:P577 ?date.

    FILTER(?date<?datef).

    ?f sc:name ?name.
    ?actor sc:name ?aName.
} 
GROUP BY ?f ?name ?actor  ?aName
ORDER BY DESC(?films)
"""

print("Results")
x=run_query(queryString)

Results
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q314914'), ('aName', 'Donald Pleasence'), ('films', '80')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q3093664'), ('aName', 'Gabriel Cattand'), ('films', '25')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q2865227'), ('aName', 'Arthur Malet'), ('films', '12')]
[('f', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('actor', 'http://www.wikidata.org/entity/Q263765'), ('aName', 'Betsy Palmer'), ('films', '7')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q1064022'), ('aName', 'Charles Cyphers'), ('films', '7')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q270079')

In [18]:
### insert the result of TASK 2 in the file
halloween_uri = "http://www.wikidata.org/entity/Q221103"
halloween_name = "Halloween"
friday_uri = "http://www.wikidata.org/entity/Q1243029"
friday_name = "Friday the 13th"
pleasence_uri = "http://www.wikidata.org/entity/Q314914"
pleasence_name = "Donald Pleasence"
palmer_uri = "http://www.wikidata.org/entity/Q2865227"
palmer_name = "Betsy Palmer"

halloween_res = {"uri":pleasence_uri,"name":pleasence_name,"check":["uri","name"],"refers_to":halloween_uri,"refers_to_name":halloween_name}
friday_res = {"uri":palmer_uri,"name":palmer_name,"check":["uri","name"],"refers_to":friday_uri,"refers_to_name":friday_name}
obj = [halloween_res,friday_res]
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_REFERRED ,["uri","name"], obj)

The index of this workflow is: 5_2
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_2.json
JSON object updated


### Task 3: Check which actor/actress did more movies after the release of these two films (consider the films separately).

Retrieve who acted in more films after the release of Friday 13th.

In [13]:
queryString = """
SELECT ?f ?name ?actor ?aName (COUNT(DISTINCT ?film) AS ?films)
WHERE { 
    VALUES ?f {wd:Q1243029 wd:Q221103}.
    ?f wdt:P161 ?actor;
                wdt:P577 ?datef.
    ?film wdt:P161 ?actor;
          wdt:P577 ?date.

    FILTER(?date>?datef).

    ?f sc:name ?name.
    ?actor sc:name ?aName.
} 
GROUP BY ?f ?name ?actor  ?aName
ORDER BY DESC(?films)

"""

print("Results")
x=run_query(queryString)

Results
[('f', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th'), ('actor', 'http://www.wikidata.org/entity/Q3454165'), ('aName', 'Kevin Bacon'), ('films', '64')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q314914'), ('aName', 'Donald Pleasence'), ('films', '55')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q106997'), ('aName', 'Jamie Lee Curtis'), ('films', '52')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q270079'), ('aName', 'P. J. Soles'), ('films', '20')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q95008'), ('aName', 'John Carpenter'), ('films', '18')]
[('f', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('actor', 'http://www.wikidata.org/entity/Q3093664'), 

In [19]:
### insert the result of TASK 3 in the file
halloween_uri = "http://www.wikidata.org/entity/Q221103"
halloween_name = "Halloween"
friday_uri = "http://www.wikidata.org/entity/Q1243029"
friday_name = "Friday the 13th"
bacon_uri = "http://www.wikidata.org/entity/Q3454165"
bacon_name = "Kevin Bacon"
pleasence_uri = "http://www.wikidata.org/entity/Q314914"
pleasence_name = "Donald Pleasence"

halloween_res = {"uri":pleasence_uri,"name":pleasence_name,"check":["uri","name"],"refers_to":halloween_uri,"refers_to_name":halloween_name}
friday_res = {"uri":bacon_uri,"name":bacon_name,"check":["uri","name"],"refers_to":friday_uri,"refers_to_name":friday_name}
obj = [halloween_res,friday_res]
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_REFERRED ,["uri","name"], obj)

The index of this workflow is: 5_2
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_2.json
JSON object updated


### Task 4: Get all the movies of the two franchises (return year and director)

`wdt:P179` is the predicate part of the series, that is useful to find the main entity. 

In [15]:
queryString = """
SELECT DISTINCT ?franchise ?name WHERE{
    
    VALUES ?f {wd:Q221103 wd:Q1243029 }.
    ?f wdt:P179 ?franchise.
    ?franchise sc:name ?name .

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('name', 'Halloween')]
[('franchise', 'http://www.wikidata.org/entity/Q1138878'), ('name', 'Friday the 13th')]
2


Since a film can have more than one director and more than one year of release (usually it depends by the country of release), in this query I decide to get as result set all triples **film-director-year** instead of using the group concat 

In [21]:
queryString = """
SELECT DISTINCT ?film ?fName ?dir ?director ?year WHERE{
    
    VALUES ?f {wd:Q221103 wd:Q1243029 }.
    ?f wdt:P179 ?o.
    ?film wdt:P179 ?o;
          wdt:P57  ?dir;
          wdt:P577 ?date.
    BIND(year(xsd:dateTime(?date)) as ?year).
    #Labels
    ?dir sc:name ?director .
    ?film sc:name ?fName .

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('fName', 'Friday the 13th Part VII: The New Blood'), ('dir', 'http://www.wikidata.org/entity/Q3052099'), ('director', 'John Carl Buechler'), ('year', '1988')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('fName', 'Halloween III: Season of the Witch'), ('dir', 'http://www.wikidata.org/entity/Q574983'), ('director', 'Tommy Lee Wallace'), ('year', '1982')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('fName', 'Halloween'), ('dir', 'http://www.wikidata.org/entity/Q95008'), ('director', 'John Carpenter'), ('year', '1978')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('fName', 'Halloween'), ('dir', 'http://www.wikidata.org/entity/Q95008'), ('director', 'John Carpenter'), ('year', '1979')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('fName', 'Halloween II'), ('dir', 'http://www.wikidata.org/entity/Q95008'), ('director', 'John Carpenter'), ('year', '1981')]
[('film', 'http://www.wikidata.org/ent

In [22]:
objs=[]
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    dir_uri = i[2][1]
    dir_name = i[3][1]
    year = i[4][1]
    # add the year element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"] = year
    objs.append(obj)
    # add the director element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"] = dir_uri
    obj["name"] = dir_name
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,["uri","name","value"], objs, elements_per_tuple = 2)

The index of this workflow is: 5_2
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_2.json
JSON object updated


### Task 5: Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)?

Retrieve the most involved worker in both franchises.

In [20]:
queryString = """
SELECT ?franchise ?fName ?worker ?wName (COUNT(DISTINCT ?film) AS ?Appearances)
{  
    VALUES ?f {wd:Q221103 wd:Q1243029 }.
    ?f wdt:P179 ?franchise.
    ?film wdt:P179 ?franchise. #franchise
    ?film ?p ?worker.
    ?worker wdt:P31 wd:Q5 ;
            wdt:P106/wdt:P31 wd:Q4220920 .
    
    #Labels
    ?worker sc:name ?wName.
    ?franchise sc:name ?fName .
}
GROUP BY ?franchise ?fName ?worker ?wName
ORDER BY DESC(?Appearances)
LIMIT 6
"""
print("Results")
x=run_query(queryString)

Results
[('franchise', 'http://www.wikidata.org/entity/Q1138878'), ('fName', 'Friday the 13th'), ('worker', 'http://www.wikidata.org/entity/Q664025'), ('wName', 'Harry Manfredini'), ('Appearances', '9')]
[('franchise', 'http://www.wikidata.org/entity/Q1138878'), ('fName', 'Friday the 13th'), ('worker', 'http://www.wikidata.org/entity/Q709325'), ('wName', 'Sean S. Cunningham'), ('Appearances', '6')]
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('fName', 'Halloween'), ('worker', 'http://www.wikidata.org/entity/Q380471'), ('wName', 'Moustapha Akkad'), ('Appearances', '5')]
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('fName', 'Halloween'), ('worker', 'http://www.wikidata.org/entity/Q314914'), ('wName', 'Donald Pleasence'), ('Appearances', '5')]
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('fName', 'Halloween'), ('worker', 'http://www.wikidata.org/entity/Q106997'), ('wName', 'Jamie Lee Curtis'), ('Appearances', '5')]
[('franchise', 'http://www.wi

In [11]:
### insert the result of TASK 5 in the file
halloween_uri = "http://www.wikidata.org/entity/Q1364022"
halloween_name = "Halloween"
friday_uri = "http://www.wikidata.org/entity/Q1138878"
friday_name = "Friday the 13th"
manfredini_uri = "http://www.wikidata.org/entity/Q664025"
manfredini_name = "Harry Manfredini"
pleasence_uri = "http://www.wikidata.org/entity/Q314914"
pleasence_name = "Donald Pleasence"

halloween_res = {"uri":pleasence_uri,"name":pleasence_name,"check":["uri","name"],"refers_to":halloween_uri,"refers_to_name":halloween_name}
friday_res = {"uri":manfredini_uri,"name":manfredini_name,"check":["uri","name"],"refers_to":friday_uri,"refers_to_name":friday_name}
obj = [halloween_res,friday_res]
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,["uri","name"], obj)

The index of this workflow is: 5_2
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_2.json
JSON object updated


### Task 6: Investigate which franchise is highly regarded by the critics overall

Since this Wikidata snapshot does not contains the review score I can use the total number of awards won by the franchises

In [22]:
queryString = """
SELECT DISTINCT ?franchise ?name (COUNT(?o) AS ?wins) (COUNT(?x) AS ?nominations)
{ 
    VALUES ?f {wd:Q221103 wd:Q1243029 }.
    ?f wdt:P179 ?franchise.
    ?film wdt:P179 ?franchise. #franchise
    OPTIONAL{?film wdt:P166 ?o.}
    OPTIONAL{?film wdt:P1411 ?x.}
    
    ?franchise sc:name ?name
}
GROUP BY ?franchise ?name
"""
print("Results")
x=run_query(queryString)

Results
[('franchise', 'http://www.wikidata.org/entity/Q1138878'), ('name', 'Friday the 13th'), ('wins', '0'), ('nominations', '2')]
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('name', 'Halloween'), ('wins', '1'), ('nominations', '0')]
2


#### TASK 6 considerations
I used the number of nominations and awards won to measure how much the crititcs liked the films belonging to the two franchises. 
Friday 13th has been nominated two times for Golden Raspberry Award. Unexpectedly this is not a good thing because it was for worst picture and worst
actress, so I'd say that critics liked Halloween franchise more than Friday 13th one.

### Task 7: Investigate which franchise costed more 

Check how many movies have cost (P2130) and bugdet (P2769).

In [23]:
#check how many movies have cost (P2130)
queryString = """
SELECT ?film ?name ?cost WHERE
{  
    VALUES ?franchise {wd:Q1364022 wd:Q1138878}.
    ?film wdt:P179 ?franchise.
    ?film wdt:P2130 ?cost.
    ?film <http://schema.org/name> ?name .

}
ORDER BY DESC(?cost)
"""
print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1454799'), ('name', 'Friday the 13th Part VI: Jason Lives'), ('cost', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('name', 'Friday the 13th Part VII: The New Blood'), ('cost', '2800000')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('name', 'Friday the 13th: The Final Chapter'), ('cost', '2600000')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('name', 'Friday the 13th Part III'), ('cost', '2250000')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('name', 'Friday the 13th Part 2'), ('cost', '1250000')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('cost', '325000')]
6


In [24]:
#check how many movies have bugdet (P2769)
queryString = """
SELECT ?film ?name ?budget WHERE
{  
    VALUES ?franchise {wd:Q1364022 wd:Q1138878}.
    ?film wdt:P179 ?franchise.
    ?film wdt:P2769 ?budget.
    ?film <http://schema.org/name> ?name .

}
ORDER BY DESC(?budget)
"""
print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q176488'), ('name', 'Halloween: Resurrection'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('name', 'Halloween'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('name', 'Halloween: The Curse of Michael Myers'), ('budget', '5000000')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('name', 'Halloween 5: The Revenge of Michael Myers'), ('budget', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('name', 'Halloween III: Season of the Witch'), ('budget', '2500000')]
5


The two sets of result are disjoint, so we can take the union

In [25]:
# print movie with cost/budget
queryString = """
SELECT ?film ?name ?cost ?budget WHERE
{  
    VALUES ?franchise {wd:Q1364022 wd:Q1138878}.
    ?film wdt:P179 ?franchise.
    {{?film wdt:P2130 ?cost.} UNION {?film wdt:P2769 ?budget.}}
    ?film <http://schema.org/name> ?name .

}
ORDER BY DESC (?cost) DESC (?budget)
"""
print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1454799'), ('name', 'Friday the 13th Part VI: Jason Lives'), ('cost', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('name', 'Friday the 13th Part VII: The New Blood'), ('cost', '2800000')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('name', 'Friday the 13th: The Final Chapter'), ('cost', '2600000')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('name', 'Friday the 13th Part III'), ('cost', '2250000')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('name', 'Friday the 13th Part 2'), ('cost', '1250000')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('name', 'Halloween'), ('cost', '325000')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('name', 'Halloween: Resurrection'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('name', 'Halloween'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('name', 'Halloween: The Cur

In [26]:
# final result
queryString = """
SELECT ?franchise ?name (SUM(?cost) AS ?t_cost) (SUM(?budget) AS ?t_budget) WHERE
{  
    VALUES ?franchise {wd:Q1364022 wd:Q1138878}.
    ?film wdt:P179 ?franchise.
    {{?film wdt:P2130 ?cost.} UNION {?film wdt:P2769 ?budget.}}
    ?franchise sc:name ?name .

}
GROUP BY ?franchise ?name
"""
print("Results")
x=run_query(queryString)

Results
[('franchise', 'http://www.wikidata.org/entity/Q1138878'), ('name', 'Friday the 13th'), ('t_cost', '11900000')]
[('franchise', 'http://www.wikidata.org/entity/Q1364022'), ('name', 'Halloween'), ('t_cost', '325000'), ('t_budget', '40500000')]
2


In [27]:
### insert the result of TASK 7 in the file
obj1 = {"refers_to":x[0][0][1],"check":"value","refers_to_name":x[0][1][1],"value":x[0][2][1]}
obj2 = {"refers_to":x[1][0][1],"check":"value","refers_to_name":x[1][1][1],"value":x[1][3][1]}
obj = [obj1,obj2]

add_result("7", TYPE_REFERRED ,"value",obj)

JSON object updated
